In [ ]:
# Modules
import inspect
import numpy as np
import pandas as pd
import seaborn as sns
import inspect
import warnings

from sklearn import ensemble, metrics, model_selection, preprocessing, tree
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error, median_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.inspection import PartialDependenceDisplay

# Suppress all warnings
warnings.filterwarnings('ignore')

## XG Boosted Tree

In [ ]:
# Import the data
df_selected = pd.read_csv('Data_selection.csv', index_col=0)

##### 1.1 Initial Fitting on data set with preliminary feature selection

In [ ]:
df = df_selected

# Randomly split the data set into training and testing and deal with the imbalanced dependent variable using SMOTE
y = df['TARGET']
X = df.drop('TARGET', axis=1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size = 0.75, shuffle = True, random_state = 480)

undersampler = RandomUnderSampler(random_state=480)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

X_train_resampled.columns = [col.replace('[', '').replace(']', '').replace('<', '') for col in X_train_resampled.columns]
X_test.columns = [col.replace('[', '').replace(']', '').replace('<', '') for col in X_test.columns]

# Fit an XGBoost model with 25 trees to the training data
XGBModel = XGBClassifier(n_estimators=25, random_state=480)
XGBModel.fit(X_train_resampled, y_train_resampled)

# make predictions on the training and test sets
y_pred_train = XGBModel.predict(X_train_resampled)
y_pred_test = XGBModel.predict(X_test)

# compute in-sample performance metrics
accuracy_train = metrics.accuracy_score(y_train_resampled, y_pred_train)
precision_train = metrics.precision_score(y_train_resampled, y_pred_train)
recall_train = metrics.recall_score(y_train_resampled, y_pred_train)
f1_score_train = metrics.f1_score(y_train_resampled, y_pred_train)
roc_auc_train = metrics.roc_auc_score(y_train_resampled, y_pred_train)

# compute out-of-sample performance metrics
accuracy_test = metrics.accuracy_score(y_test, y_pred_test)
precision_test = metrics.precision_score(y_test, y_pred_test)
recall_test = metrics.recall_score(y_test, y_pred_test)
f1_score_test = metrics.f1_score(y_test, y_pred_test)
roc_auc_test = metrics.roc_auc_score(y_test, y_pred_test)

# assuming you have already computed the performance metrics
data = {'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC AUC'],
        'Training': [accuracy_train, precision_train, recall_train, f1_score_train, roc_auc_train],
        'Test': [accuracy_test, precision_test, recall_test, f1_score_test, roc_auc_test]}
df = pd.DataFrame(data)
df

In [ ]:
rmse_train = mean_squared_error(y_train_resampled, y_pred_train, squared = False)
mae_train = mean_absolute_error(y_train_resampled, y_pred_train)

rmse_test = mean_squared_error(y_test, y_pred_test, squared = False)
mae_test = mean_absolute_error(y_test, y_pred_test)

data = {'Metric': ['RMSE', 'MAE'],
        'Training': [rmse_train, mae_train],
        'Test': [rmse_test, mae_test]}
df = pd.DataFrame(data)
df

##### 1.2 Hyperparameter Tuning using Cross-validation on data set with preliminary feature selection

In [ ]:
# define the parameter grid
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.8, 1],
    'colsample_bytree': [0.5, 0.8, 1]
}

# create an XGBoost model
xgb = XGBClassifier(random_state=480, max_depth = 3, learning_rate = 0.1, n_estimators = 300, gamma = 0, min_child_weight = 1, subsample = 1, colsample_bytree = 1)

# create a grid search object
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5)

# fit the grid search object to the data
grid_search.fit(X_train_resampled, y_train_resampled)

print(grid_search.best_params_)

##### 1.3 Refit the model using optimal parameters found through hyperparameter tuning on data set with preliminary feature selection

In [ ]:
rf = XGBClassifier(random_state=480, max_depth = 3, learning_rate = 0.1, n_estimators = 300, gamma = 0, min_child_weight = 1, subsample = 1, colsample_bytree = 1)

# fit the model to the training data
rf.fit(X_train_resampled, y_train_resampled)

# make predictions on the training and test sets
y_pred_train = rf.predict(X_train_resampled)
y_pred_test = rf.predict(X_test)

# compute in-sample performance metrics
accuracy_train = metrics.accuracy_score(y_train_resampled, y_pred_train)
precision_train = metrics.precision_score(y_train_resampled, y_pred_train)
recall_train = metrics.recall_score(y_train_resampled, y_pred_train)
f1_score_train = metrics.f1_score(y_train_resampled, y_pred_train)
roc_auc_train = metrics.roc_auc_score(y_train_resampled, y_pred_train)

# compute out-of-sample performance metrics
accuracy_test = metrics.accuracy_score(y_test, y_pred_test)
precision_test = metrics.precision_score(y_test, y_pred_test)
recall_test = metrics.recall_score(y_test, y_pred_test)
f1_score_test = metrics.f1_score(y_test, y_pred_test)
roc_auc_test = metrics.roc_auc_score(y_test, y_pred_test)

# assuming you have already computed the performance metrics
data = {'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC AUC'],
        'Training': [accuracy_train, precision_train, recall_train, f1_score_train, roc_auc_train],
        'Test': [accuracy_test, precision_test, recall_test, f1_score_test, roc_auc_test]}
df = pd.DataFrame(data)
df

In [ ]:
rmse_train = mean_squared_error(y_train_resampled, y_pred_train, squared = False)
mae_train = mean_absolute_error(y_train_resampled, y_pred_train)

rmse_test = mean_squared_error(y_test, y_pred_test, squared = False)
mae_test = mean_absolute_error(y_test, y_pred_test)

data = {'Metric': ['RMSE', 'MAE'],
        'Training': [rmse_train, mae_train],
        'Test': [rmse_test, mae_test]}
df = pd.DataFrame(data)
df

In [ ]:
# Get and reshape confusion matrix data
matrix = confusion_matrix(y_test, y_pred_test)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
pyplot.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=pyplot.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['TARGET = 0', 'TARGET = 1']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
pyplot.xticks(tick_marks, class_names, rotation=25)
pyplot.yticks(tick_marks2, class_names, rotation=0)
pyplot.xlabel('Predicted label')
pyplot.ylabel('True label')
pyplot.title('Confusion Matrix for Random Forest Model')
pyplot.show()

In [ ]:
# Calculate feature importance
importances = rf.feature_importances_

# Create a DataFrame to store the results
importance_df = pd.DataFrame({'Feature': X_train_resampled.columns, 'Importance': importances})

# Sort the DataFrame by importance and select the top 10 features
importance_df = importance_df.sort_values(by='Importance', ascending=False).head(10)

# Display the results
importance_df

In [ ]:
# get feature importances
importances = rf.feature_importances_

# get indices of top 10 features
indices = np.argsort(importances)[::-1][:10]

# calculate partial dependence
fig, ax = pyplot.subplots(5, 2, figsize=(20, 20))
pdp_results = PartialDependenceDisplay.from_estimator(rf, X_train_resampled, indices,
                                                      feature_names=X_train_resampled.columns,
                                                      ax=ax.ravel())
# reduce the font size of the titles
for ax in pdp_results.axes_.ravel():
    ax.set_title(ax.get_title(), fontsize=5)
    
pyplot.tight_layout()